In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Store CSV into DataFrame

In [2]:
csv_file = "Resources/raw_state_automation_data_1.csv"
raw_state_automation_data_df = pd.read_csv(csv_file)
raw_state_automation_data_df.head()

,SOC,Occupation,Probability,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,...,South_Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West_Virginia,Wisconsin,Wyoming
0,11-1011,Chief Executives,0.015,1030,760,5750,2710,31150,880,1410,...,560,5460,5890,3650,280,6320,5910,980,3740,160
1,11-1021,General and Operations Managers,0.160,26930,6490,43300,20680,261780,41540,33280,...,3730,44400,168610,36200,2760,52380,43760,10200,32350,4840
2,11-3061,Purchasing Managers,0.030,950,130,1780,510,8460,760,1360,...,50,2000,5030,450,150,2480,2010,80,1290,40
3,17-3013,Mechanical Drafters,0.680,590,0,750,240,3900,880,1900,...,170,1240,4900,650,90,2020,2250,150,2730,80
4,17-3021,Aerospace Engineering and Operations Technicians,0.480,190,0,570,60,2490,310,150,...,0,0,1200,130,0,160,0,0,0,0


# Create new data with select columns

In [3]:
state_automation_data_df = raw_state_automation_data_df[['SOC', 'Occupation', 'Probability']].copy()

# Rename the column headers
state_automation_data_df = state_automation_data_df.rename(columns={"SOC": "soc",
                                                        "Occupation": "occupation",
                                                        "Probability": "probability"})




state_automation_data_df.set_index("soc", inplace=True)
state_automation_data_df.head()

,occupation,probability
soc,,
11-1011,Chief Executives,0.015
11-1021,General and Operations Managers,0.160
11-3061,Purchasing Managers,0.030
17-3013,Mechanical Drafters,0.680
17-3021,Aerospace Engineering and Operations Technicians,0.480


In [4]:
csv_file = "Resources/occupational_employment_statistics_1.csv"
occupational_employment_statistics_data_df = pd.read_csv(csv_file)
occupational_employment_statistics_data_df.head()

,Area_Type,Area,Area_Name,SOC,Occupational_Title,Employment,Mean_Wage,Median_Wage,Entry_Wage,Experienced_Wage
0,10,1,Capital Region,11-2031,Public Relations and Fundraising Managers,400.0,103400,89550.0,55480.0,127360.0
1,10,6,Mohawk Valley Region,15-1111,Computer and Information Research Scientists,120.0,103680,102530.0,79840.0,115610.0
2,10,10,Western New York Region,39-4031,"Morticians, Undertakers, and Funeral Directors",NaN,62910,64990.0,45420.0,71660.0
3,10,8,North Country Region,39-9099,"Personal Care and Service Workers, All Other",NaN,30210,30430.0,24730.0,32950.0
4,10,10,Western New York Region,33-3041,Parking Enforcement Workers,NaN,35380,35320.0,29380.0,38390.0


In [5]:
employment_wage_df = occupational_employment_statistics_data_df[['SOC', 'Occupational_Title', 'Mean_Wage', 'Median_Wage', 'Entry_Wage', 'Experienced_Wage']].copy()                                                                                                                  


employment_wage_df = employment_wage_df.rename(columns={"SOC": "soc",
                                                        "Occupational_Title": "occupation",
                                                        "Mean_Wage": "mean_wage",
                                                        "Median_Wage": "median_wage",
                                                        "Entry_Wage": "entry_wage", 
                                                        "Experienced_Wage": "experienced_wage"})

employment_wage_df.set_index("soc", inplace=True)
employment_wage_df.head()

,occupation,mean_wage,median_wage,entry_wage,experienced_wage
soc,,,,,
11-2031,Public Relations and Fundraising Managers,103400,89550.0,55480.0,127360.0
15-1111,Computer and Information Research Scientists,103680,102530.0,79840.0,115610.0
39-4031,"Morticians, Undertakers, and Funeral Directors",62910,64990.0,45420.0,71660.0
39-9099,"Personal Care and Service Workers, All Other",30210,30430.0,24730.0,32950.0
33-3041,Parking Enforcement Workers,35380,35320.0,29380.0,38390.0


In [6]:
rds_connection_string = "postgres:Darren2019@localhost:5432/occupation_automation_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [7]:
engine.table_names()

['automation_probability', 'occupational_employment_statistics']

# Load DataFrames into Database

In [13]:
state_automation_data_df.to_sql(name='automation_probability', con=engine, if_exists='append', index=True)

In [14]:
employment_wage_df.to_sql(name='occupational_employment_statistics', con=engine, if_exists='append', index=True)

# Confirm data has been added by querying the table

In [15]:
pd.read_sql_query('select * from automation_probability', con=engine).head()

,soc,occupation,probability
0,11-1011,Chief Executives,0.015
1,11-1021,General and Operations Managers,0.160
2,11-3061,Purchasing Managers,0.030
3,17-3013,Mechanical Drafters,0.680
4,17-3021,Aerospace Engineering and Operations Technicians,0.480


In [16]:
pd.read_sql_query('select * from occupational_employment_statistics', con=engine).head()

,soc,occupation,mean_wage,median_wage,entry_wage,experienced_wage
0,11-2031,Public Relations and Fundraising Managers,103400,89550.0,55480.0,127360.0
1,15-1111,Computer and Information Research Scientists,103680,102530.0,79840.0,115610.0
2,39-4031,"Morticians, Undertakers, and Funeral Directors",62910,64990.0,45420.0,71660.0
3,39-9099,"Personal Care and Service Workers, All Other",30210,30430.0,24730.0,32950.0
4,33-3041,Parking Enforcement Workers,35380,35320.0,29380.0,38390.0
